# ConvLSTM

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model, Sequential
from tensorflow.keras import backend as K

AttributeError: module 'googleapiclient' has no attribute '__version__'

In [ ]:
print (tf.__version__)

Let's just make a model and experiment with it

In [ ]:
frames = 6
channels = 1
pixels_x = 21
pixels_y = 21

## Model architecture: encoder/decoder

NOTE: Tensorflow and Keras don't have support for initial_state on ConvLSTM2D cells. 
Please follow the instructions here: https://stackoverflow.com/questions/50253138/convlstm2d-initial-state-assertion-error to fix it.

### encoder model for fixed slice size

In [ ]:
##### ENCODER #####

# first time-step
i = 0
input1 = layers.Input(name="encoder_input{}".format(i+1),
                      shape = (1, channels, pixels_x, pixels_y))
encoder_cell_1 = layers.ConvLSTM2D(name="encoder{}".format(i+1),
                            filters = 1, kernel_size=(5,5), padding='same',
                            data_format='channels_first', return_sequences=True ,return_state=True)
_, state_h, state_c = encoder_cell_1(input1)

encoder_states = [state_h, state_c]

##### DECODER #####

input2 = layers.Input(name="decoder_input{}".format(i+1),
                      shape = (1, channels, pixels_x, pixels_y))
decoder_cell_1 = layers.ConvLSTM2D(name="decoder{}".format(i+1),
                            filters = 1, kernel_size=(5,5), padding='same',
                            data_format='channels_first', return_sequences=True ,return_state=True)
decoder_output, _, _ = decoder_cell_1(input1, initial_state = encoder_states)

##### COLLECT AND COMPILE #####
encoder_stack = Model(inputs = [input1, input2], 
                      outputs = decoder_output
                     )

encoder_stack.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['mean_absolute_error'])
encoder_stack.summary()

### for loop to create for various slice sizes

## create generator to feed inputs to model

In [ ]:
def get_array_slices_proc(array):
    """
    Switch first and second dimensions of array and reshape to add empty dim in front. 
    Must know frames, channels, pixels_x, and pixels_y
    """
    # switch first and second axes (in practice from (channels, frames, pixels_x, pixels_y) to (frames, channels, pixels_x, pixels_y) )
    array = np.moveaxis(array, 0, 1)
    # add empty dimension in front for ConvLSTMs
    array.reshape(-1, 1, channels, pixels_x, pixels_y)
    return array

In [ ]:
import glob
import xarray as xr
def generate_arrays(img_dir, slice_size=24, vars_=['t2m']):
    """
    A generator that returns a list of slices (length = slice_size) as input, and another list of the subsequent 24-hour slice as output
    """
    # get list of netcdf files in img_dir
    netcdf_dirs = sorted(glob.glob(img_dir+"/*.nc"))
    file_index = 0
    # open first netcdf file
    ds = xr.open_dataset(netcdf_dirs[file_index])
    # select only some variables
    ds = ds[vars_]
    # counter is for hourly time slices. months with 31 days have 744 hours
    counter = 0
    while True: # generator needs to run infinitely
        
        # get input slice
        input_images = []
        output_images = []
        for i in range(slice_size, 0, -1):
            ds_slice = ds.isel(time=slice(counter+i, counter+i+1)).to_array().values
            input_image_slice = get_array_slices_proc(ds_slice)
            input_images.append(input_image_slice)
        
        # check if we're at the end of the month
        if counter+2*slice_size > ds.sizes['time']:
            # reset slice counter, increment to next netcdf file, open it, get output images
            counter = 0
            file_index += 1
            if file_index == len(netcdf_dirs):
                file_index=0
                
            ds = xr.open_dataset(netcdf_dirs[file_index])
            # select only some variables
            ds = ds[vars_]
            # take slice 0-24 as output-image
            for i in range(slice_size, 0, -1):
                ds_slice = ds.isel(time=slice(counter+i, counter+i+1)).to_array().values
                output_image_slice = get_array_slices_proc(ds_slice)
                output_images.append(input_image_slice)
            # set counter to -slice_size to reset for input on next iteration
            counter -= slice_size
        # get output slice right after input slice
        else:
            for i in range(slice_size, 0, -1):
                ds_slice = ds.isel(time=slice(counter+slice_size+i, counter+slice_size+i+1)).to_array().values
                output_image_slice = get_array_slices_proc(ds_slice)
                output_images.append(input_image_slice)
        
        yield (input_images, output_images)
        counter += slice_size

In [ ]:
train_file_path = "../data/train"
validate_file_path = "../data/validate"

# 3 years of training data = 
train_steps = 3 * 365 * 24 / 6 - 1
# 1 year of validation data = 
valid_steps = 1 * 365 * 24 / 6 - 1

In [ ]:
gen = generate_arrays(train_file_path, slice_size=6)

In [ ]:
in_, out_ = next(gen)

In [ ]:
out_.shape

In [ ]:
history = encoder_stack.fit_generator(
    generate_arrays(train_file_path, slice_size=1),
    steps_per_epoch = train_steps,
    epochs = 20,
    verbose = 1,
    shuffle = False,
    initial_epoch = 0,
    validation_steps = valid_steps,
    validation_data = generate_arrays(validate_file_path, slice_size=1),
    )

In [ ]:
import pickle
import datetime
current_datetime = datetime.datetime.now().strftime("%Y_%M_%d_%H%M")
pickle.dump( history, open( "../models/"+current_datetime+"_convlstm.pkl", "wb" ) )

## Functional API model version

In [ ]:
#rewrite model in Functional style

# main input
frame_input = layers.Input(shape=(frames, channels, pixels_x, pixels_y), name='frame_input')


# ConvLSTM block
stack = layers.ConvLSTM2D(filters=20, kernel_size=(5,5), padding='same',
                       data_format='channels_first',return_sequences=True, name='ConvLSTM_1')(frame_input)
stack = layers.BatchNormalization(axis=1, name='batchnorm_1')(stack)
stack = layers.ConvLSTM2D(filters=10, kernel_size=(5,5), padding='same', 
                       data_format='channels_first',return_sequences=True, name='ConvLSTM_2')(stack)
stack = layers.BatchNormalization(axis=1, name='batchnorm_2')(stack)
stack = layers.ConvLSTM2D(filters=10, kernel_size=(1,1), padding='same', return_state=True,
                       data_format='channels_first',return_sequences=True, name='ConvLSTM_3')(stack)


# auxiliary input
# auxiliary_input = layers.Input(shape=(5,), name='aux_input')
# dense_input = layers.concatenate([clstm_out, auxiliary_input], name='concatenate_layer')




model2 = Model(inputs = [frame_input], outputs = [stack])

model2.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['mean_absolute_error'])
model2.summary()